# **Q4**
Implement logistic regression (LOR), LOR with L2-norm regularization, and LOR
with L1-norm regularization models using BGD, SGD, and MBGD algorithms. The
dataset in data_q4_q5.xlsx contains 30 features and one output. The class label ‘M’
stands for malignant, and ‘B’ is the Benign class. You must use hold-out
cross-validation ((CV) with 70% as training, 10% as validation and 20% as testing) to
evaluate training, validation, and testing instances for each model. Evaluate the
performance of each model using accuracy, sensitivity, and specificity measures.

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded=files.upload()

Saving data_q4_q5.xlsx to data_q4_q5.xlsx


In [ ]:
def hyp(x,w):
  return np.dot(x,w)
def hyp_lor(x,w):
  return 1/(1+np.exp(-(hyp(x,w))))
def mse(y,pred,l):
  return (1/l)*np.sum(np.square(y-pred))
def sensitivity(TN,FP,FN,TP):
  return TP/(TP+FN)
def specificity(TN,FP,FN,TP):
  return TN/(TN+FP)
def accuracy(TN,FP,FN,TP):
  return (TP+TN)/(TP+TN+FP+FN)

In [ ]:
data=pd.read_excel("/content/data_q4_q5.xlsx")
f,tr,va,te = 31,398,57,114
data.insert(0,"x0",1)
data.insert(f,"diag_bool",0)
dbool = (data.diagnosis=='M')
data.loc[dbool,'diag_bool']=1
data=data.drop(['diagnosis'],axis=1)

In [ ]:
data_tr=data.loc[:397,:].copy()
data_tr=np.array(data_tr)
norm = np.amax(np.abs(data_tr),axis=0)
data_va=data.loc[398:454,:].copy()
data_va=np.array(data_va)
data_te=data.loc[455:,:].copy()
data_te=np.array(data_te)
data_tr,data_va,data_te = data_tr/norm,data_va/norm,data_te/norm #normalising data
x_train,y_train,x_val,y_val,x_test,y_test=data_tr[:,:f],data_tr[:,f].reshape(tr,1),data_va[:,:f],data_va[:,f].reshape(va,1),data_te[:,:f],data_te[:,f].reshape(te,1)

## BGD

### LoR

In [ ]:
# for grid search
T_grid=np.linspace(300,950,10)
A_grid=np.linspace(0.1,1,10)
params=[0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    w=np.random.rand(f).reshape(f,1) # weight vector initialised
    for t in range(int(T)):
      j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
      for j in range(f):
        jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
        w[j] = w[j] - (1/tr)*alpha*jsum
    mse_val = mse(hyp_lor(x_val,w),y_val,va)
    if mse_val < curr_err:
      curr_err = mse_val
      params = [int(T),alpha]

In [ ]:
T_ideal,Alpha=params[0],params[1]
params

[300, 1.0]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(T_ideal):
  j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w[j] = w[j] - (1/tr)*Alpha*jsum

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.9230769230769231
Specificity = 0.9886363636363636
Accuracy = 0.9736842105263158


### LoR with L2 NORM

In [ ]:
# for grid search
T_grid=np.linspace(300,950,10)
A_grid=np.linspace(0.1,1,10)
L_grid=np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
          w[j] = (1-(lamb*alpha))*w[j] - (1/tr)*alpha*jsum
      mse_val = mse(hyp_lor(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[372, 0.9, 0.01]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(T_ideal):
  j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w[j] = (1-(Lambda*Alpha))*w[j] - (1/tr)*Alpha*jsum

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.8846153846153846
Specificity = 0.9886363636363636
Accuracy = 0.9649122807017544


### LoR with L1 NORM

In [ ]:
# for grid search
T_grid=np.linspace(300,950,10)
A_grid=np.linspace(0.1,1,10)
L_grid=np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
          w[j] = w[j] - (1/tr)*alpha*jsum - ((alpha*lamb)/2)*np.sign(w[j])
      mse_val = mse(hyp_lor(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[877, 0.30000000000000004, 0.020000000000000004]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(T_ideal):
    j1 = hyp_lor(x_train,w)-y_train # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[:,j].reshape(tr,1))
      jsum = np.sum(j2)
      w[j] = w[j] - (1/tr)*Alpha*jsum - ((Alpha*Lambda)/2)*np.sign(w[j])

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
M [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
M [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
M [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.9230769230769231
Specificity = 0.9659090909090909
Accuracy = 0.956140350877193


## SGD

### LoR

In [ ]:
# for grid search
T_grid=np.linspace(200,380,10)
A_grid=np.linspace(0.1,1,10)
params=[0,0] # T_ideal,Alpha

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    w=np.random.rand(f).reshape(f,1) # weight vector initialised
    for t in range(int(T)):
      index=np.random.randint(tr)
      j1 = (hyp_lor(x_train,w)[index]-y_train[index]).reshape(1,1) # (hyp(xi)-yi)
      for j in range(f):
        j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
        w[j] = (1-(lamb*alpha))*w[j] - alpha*j2
    mse_val = mse(hyp_lor(x_val,w),y_val,va)
    if mse_val < curr_err:
      curr_err = mse_val
      params = [int(T),alpha]

In [ ]:
T_ideal,Alpha=params[0],params[1]
params

[372, 0.6]

In [ ]:
w=np.random.rand(f).reshape(f,1)

for t in range(T_ideal):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w)[index]-y_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
    w[j] = w[j] - Alpha*j2

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)*100))
print("Specificity = "+str(specificity(TN,FP,FN,TP)*100))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)*100))

Sensitivity = 84.61538461538461
Specificity = 100.0
Accuracy = 96.49122807017544


### LoR with L2 NORM

In [ ]:
# for grid search
T_grid=np.linspace(200,400,10)
A_grid=np.linspace(0.1,1,10)
L_grid=np.linspace(0.01,0.1,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        index=np.random.randint(tr)
        j1 = (hyp_lor(x_train,w)[index]-y_train[index]).reshape(1,1) # (hyp(xi)-yi)
        for j in range(f):
          j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
          w[j] = (1-(lamb*alpha))*w[j] - alpha*j2
      mse_val = mse(hyp_lor(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[340, 1.0, 0.01]

In [ ]:
w=np.random.rand(f).reshape(f,1)

for t in range(T_ideal):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w)[index]-y_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
    w[j] = (1-(Lambda*Alpha))*w[j] - Alpha*j2

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
B [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
B [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
B [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.7307692307692307
Specificity = 1.0
Accuracy = 0.9385964912280702


### LoR with L1 NORM

In [ ]:
# for grid search
T_grid=np.linspace(50,750,15)
A_grid=np.linspace(0.01,0.7,10)
L_grid=np.linspace(0.001,0.01,10)
params=[0,0,0] # T_ideal,Alpha,Lambda

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        index=np.random.randint(tr)
        j1 = (hyp_lor(x_train,w)[index]-y_train[index]).reshape(1,1) # (hyp(xi)-yi)
        for j in range(f):
          j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
          w[j] = w[j] - (1/tr)*alpha*j2 - ((alpha*lamb)/2)*np.sign(w[j])
      mse_val = mse(hyp_lor(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[300, 0.16333333333333333, 0.006]

In [ ]:
w=np.random.rand(f).reshape(f,1)

for t in range(T_ideal):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w)[index]-y_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1.reshape(1,1),x_train[index,j])
    w[j] = w[j] - Alpha*j2 - ((Alpha*Lambda)/2)*np.sign(w[j])

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
B [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
B [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.7692307692307693
Specificity = 1.0
Accuracy = 0.9473684210526315


## MGBD

### LoR

In [ ]:
# for grid search
T_grid=np.linspace(300,560,10)
A_grid=np.linspace(0.1,1,10)
params=[0,0] # T_ideal,Alpha
nb=32

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    w=np.random.rand(f).reshape(f,1) # weight vector initialised
    for t in range(int(T)):
      np.random.shuffle(data_tr)
      hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array part1
      hypo = 1/(1+np.exp(-hypo1))
      j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
      for j in range(f):
        jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
        w[j] = w[j] - (1/tr)*alpha*jsum
    mse_val = mse(hyp_lor(x_val,w),y_val,va)
    if mse_val < curr_err:
      curr_err = mse_val
      params = [int(T),alpha]

In [ ]:
T_ideal,Alpha=params[0],params[1]
params

[502, 1.0]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(int(T_ideal)):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
    w[j] = w[j] - (1/tr)*Alpha*jsum

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
M [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.8846153846153846
Specificity = 0.9545454545454546
Accuracy = 0.9385964912280702


### LoR with L2 NORM




In [ ]:
# for grid search
T_grid=np.linspace(300,560,10)
A_grid=np.linspace(0.01,0.8,10)
L_grid=np.linspace(0.001,0.01,10)
params=[0,0,0] # T_ideal,Alpha,Lambda
nb=32

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        np.random.shuffle(data_tr)
        hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array part1
        hypo = 1/(1+np.exp(-hypo1))
        j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
          w[j] = (1-(lamb*alpha))*w[j] - (1/tr)*alpha*jsum
      mse_val = mse(hyp_lor(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[560, 0.8, 0.001]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(int(T_ideal)):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
    w[j] = (1-(Lambda*Alpha))*w[j] - (1/tr)*Alpha*jsum

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.8846153846153846
Specificity = 0.9772727272727273
Accuracy = 0.956140350877193


### LoR with L1 NORM

In [ ]:
# for grid search
T_grid=np.linspace(10,500,10)
A_grid=np.linspace(0.005,0.8,10)
L_grid=np.linspace(0.001,0.02,10)
params=[0,0,0] # T_ideal,Alpha,Lambda
nb=20

In [ ]:
curr_err=100 # current error

for T in T_grid:
  for alpha in A_grid:
    for lamb in L_grid:
      w=np.random.rand(f).reshape(f,1) # weight vector initialised
      for t in range(int(T)):
        np.random.shuffle(data_tr)
        hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array
        hypo = 1/(1+np.exp(-hypo1))
        j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
        for j in range(f):
          jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
          w[j] = w[j] - (1/tr)*alpha*jsum - ((alpha*lamb)/2)*np.sign(w[j])
      mse_val = mse(hyp(x_val,w),y_val,va)
      if mse_val < curr_err:
        curr_err = mse_val
        params = [int(T),alpha,lamb]

In [ ]:
T_ideal=params[0]
Alpha=params[1]
Lambda=params[2]
params

[173, 0.27, 0.013666666666666667]

In [ ]:
w=np.random.rand(f).reshape(f,1) # weight vector initialised

for t in range(60):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:nb,:f].reshape(nb,f),w) # hypothesis array
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:nb,f].reshape(nb,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:nb,j].reshape(nb,1)))
    w[j] = w[j] - (1/nb)*Alpha*jsum - ((Alpha*Lambda)/2)*np.sign(w[j])

In [ ]:
# testing
TN,FP,FN,TP=0,0,0,0
predicted_output = hyp_lor(x_test,w)
i=0
for inst in predicted_output:
  if inst>0.5:
    print('M',y_test[i])
    inst=1
    if inst==y_test[i]:
      TP+=1
    else:
      FP+=1
  else:
    print('B',y_test[i])
    inst=0
    if inst==y_test[i]:
      TN+=1
    else:
      FN+=1
  i+=1

B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
M [1.]
B [0.]
B [1.]
B [0.]
M [1.]
M [1.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [1.]
B [0.]
M [1.]
B [1.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
B [0.]
M [0.]
B [0.]
B [0.]
B [0.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
M [1.]
B [0.]


In [ ]:
# Performance evaluation
print("Sensitivity = "+str(sensitivity(TN,FP,FN,TP)))
print("Specificity = "+str(specificity(TN,FP,FN,TP)))
print("Accuracy = "+str(accuracy(TN,FP,FN,TP)))

Sensitivity = 0.8461538461538461
Specificity = 0.9659090909090909
Accuracy = 0.9385964912280702
